# Representacion no estructurada de canciones de los 80s

## Importacion de datos

In [ ]:
%pip install --upgrade pip
%pip install scikit-learn matplotlib pandas seaborn nltk numpy spacy gensim==3.8.3 pyLDAvis==3.4.0

In [2]:
# Importacion de las letras desde el directorio de letras
import os
import pandas as pd
import numpy as np

# Crear dataframe con las letras: Cancion, Letra y Artista
canciones = []
for file in os.listdir('letras'):
    with open(f'letras/{file}', encoding='utf-8') as f:
        cancion = ' '.join(file.split('.')[0:-1]).strip()
        nombre = ' '.join(cancion.split('-')[0:-1]).strip()
        artista = cancion.split('-')[-1].strip()
        letras = f.read()
        canciones.append([nombre, artista, letras])

canciones = pd.DataFrame(canciones, columns=['Cancion', 'Artista', 'Letra'])

canciones.to_csv('canciones.csv', index=False)

canciones.head()

,Cancion,Artista,Letra
0,A esa,Pimpinela,"L VEN AQUI, QUIERO DECIRTE ALGO...\n\nA esa, q..."
1,A quién le importa,Alaska y Dinarama,La gente me señala\n\nme apuntan con el dedo\n...
2,Amante Bandido,Miguel Bosé,Yo seré el viento que va\n\nnavegare por tu os...
3,Amor Eterno,Rocío Dúrcal,Tu eres la tristeza de mis ojos \nque lloran e...
4,Ausencia,Héctor Lavoe,Ha terminado otro capítulo en mi vida \nLa muj...


## Representacion TF-IDF

### Entendimiento y preprocesamiento de datos

In [3]:
letras = canciones['Letra'].values.tolist()
len(letras)

105

Importacion de la libreria de stopwords y definicion de una funcion para filtrar las canciones.

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
misstop=stopwords.words('spanish')+["á","Y"]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def filtrado(texto):
    filtrados=[word for word in texto if word not in misstop]
    return(filtrados)

La limpieza incluira:

**a)** pasar a minúsculas

**b)** retirar páginas web

**c)** quitar saltos de línea y tabuladores

**d)** retirar stopwords

**e)** retirar menciones

**f)** unir todo de nuevo

**g)** eliminar letras solas


In [6]:
import re
nltk.download("punkt")
nltk.download('punkt_tab')
filtradito=[]
# Eliminar menciones y páginas web
for i in range(len(letras)):
    letras[i] = re.sub("\""," ",letras[i])
    letras[i] = re.sub ("\n|\t"," ",letras[i])
    letras[i]=letras[i].lower()
    letras[i] = re.sub(r'\s+[a-zA-Z]\s+', ' ', letras[i])
    letras[i] = re.sub(r'\xad+', ' ', letras[i], flags=re.I)
    breve=nltk.tokenize.word_tokenize(letras[i],language="spanish")
    tempfilt=filtrado(breve)
    filtradito.append(tempfilt)
    
unidito=[]
for element in filtradito:
    unidito.append(" ".join(element))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
unidito[0]

'l ven aqui , quiero decirte ... , aparta , roba tiempo , alma cuerpo , ve dile ... quieres ? venga , valor , muestre cara hable frente si quiere amor ... ? , contigo va vestida princesa , , hace preguntas siempre está¡ dispuesta , , vete dile ... ? venga ... ? doy lugar ... quieres probar ? recoja mesa , lave ropa todas miserias ... quieres demostrar ? venga , juegue ... vas conseguir ? quiero ver si capaz darte cosas di ... , ... vete dile , venga ... , pone tan mal capaz hacerme volver vivir ilusiones perdidas . , hace hablar debo cosas hace tiempo das ... , puede costar hacerte feliz hora día ? , toca vivir ninguna tristeza , alegrí a. vete dile .. ? venga ... ? doy lugar ... quieres probar ? recoja mesa , lave ropa todas miserias ... quieres demostrar ? venga , juegue ... vas conseguir ? quiero ver si capaz darte cosas di ... venga ... ... doy lugar ... quieres probar ... recoja mesa lave ropa todas miserias quieres demostrar ... venga juegue ... vas conseguir ... quiero ver si ca

### Determinacion de Topicos

Importacion de librerias para topicos de sklearn

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

Estraccion de topicos

In [9]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [10]:
vectores=TfidfVectorizer(max_df=0.9,min_df=2,max_features=1000)
tfslimpio=vectores.fit_transform(unidito)
nombreslimpio=vectores.get_feature_names_out()
nmflimpio = NMF(n_components=5, random_state=1, l1_ratio=.5, init='nndsvd').fit(tfslimpio)
no_top_words=10
display_topics(nmflimpio, nombreslimpio, no_top_words)

Topic 0:
si quiero voy amor sé vos vez noche siempre nadie
Topic 1:
vida pido why toda asi así amor ahora prefiero nunca
Topic 2:
oh bajo parte playa calienta cerca aquí luna sol busco
Topic 3:
desaparecer ayer amor muralla divide mirando parado vuelve viene realidad
Topic 4:
ayudarla sacarla puedo pozo aca escucho abajo mas mirarla calla


In [11]:
len(nombreslimpio)

790

Utilizacion de conteos para LDA (Latent Dirilchet Allocation)

In [12]:
paralda = CountVectorizer(max_df=0.9, min_df=2, max_features=1000)
tflda=paralda.fit_transform(unidito)
nombreslda=paralda.get_feature_names_out()
lda = LatentDirichletAllocation(n_components=8, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tflda)
id_topico=lda.fit_transform(tflda)
display_topics(lda,nombreslda,no_top_words)

Topic 0:
vida mundo ay solo toda tan así oh si llorar
Topic 1:
amor eu debajo um rock não ayer desaparecer corazón dia
Topic 2:
oh nadie vete bis puede verás jamás hombre luna lleva
Topic 3:
na quiero amor noches vida cinco pido luna noche vez
Topic 4:
amo je lluvia si nadie fríos besos días gota non
Topic 5:
importa asi si pasa siempre nunca amor ahora manos primavera
Topic 6:
si amor sé ahora quiero puedo nunca siempre vida gente
Topic 7:
si ven mas quiero voy nene van solo amor ser


Pertenencia de cada documento a su topico

In [13]:
doc_topic = nmflimpio.transform(tfslimpio)
topic_most_pr=[]
for n in range(doc_topic.shape[0]):
    topic_most_pr.append(doc_topic[n].argmax())

In [14]:
canciones["nmf"]=topic_most_pr
ver_cerveza=canciones[["Cancion","nmf"]]
display(ver_cerveza)

,Cancion,nmf
0,A esa,0
1,A quién le importa,1
2,Amante Bandido,0
3,Amor Eterno,0
4,Ausencia,0
...,...,...
100,Y cómo es él,1
101,Yo No Te Pido La Luna,1
102,Yo te avisé,0
103,Ámame en cámara lenta,1


### Visualizacion de resultados

Gensim y pyLDAvis para ver los resultados de LDA

In [17]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [21]:
import spacy
spacy.cli.download("es")
nlp=spacy.load("es_core_news_sm")

⚠ As of spaCy v3.0, shortcuts like 'es' are deprecated. Please use the
full pipeline package name 'es_core_news_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
